# データセット作成

In [2]:
import pandas as pd

In [3]:
# データをインポートしてきてがっちゃんこする、その後シャッフル
t = pd.read_csv('../True.csv')
f = pd.read_csv('../Fake.csv')
t_new = pd.concat([t, pd.Series([0 for i in range(len(t))])], axis=1)
f_new = pd.concat([f, pd.Series([1 for i in range(len(f))])], axis=1)
total = pd.concat([t_new, f_new], axis=0)
total_new = total.sample(frac=1, random_state=0)

In [4]:
# 訓練データを作る
train = total_new[:40000]
train.to_csv('train.csv', index=False, header=False)

# テストデータを作る
test = total_new[40000:].drop(columns=[0], axis=1)
test.to_csv('test.csv', index=False, header=False)

# サンプルサブミッションを作る
sample_submission = pd.concat([pd.Series(test.index), pd.Series([0 for i in range(len(test))])], axis=1)
sample_submission.to_csv('sample_submission.csv', index=False, header=False)

# 評価指標作成

In [ ]:
def calculate_accuracy(pred_y, true_y):
    """
    精度を計算する関数
    :param pred: 予測値
    :param true: 正解値
    :return: 精度
    """

    # 予測ラベル
    y_pred = pred_y.iloc[:, 1].values
    # 正解ラベル
    y_true = true_y.iloc[:, 1].values
    # accuracyの算出
    correct = (y_pred == y_true).sum()
    total = len(y_true)
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')
    return accuracy

In [48]:
true_y = pd.concat([pd.Series(test.index), total_new[40000:][0].reset_index(drop=True)], axis=1)

In [50]:
final_metrics = calculate_accuracy(sample_submission, true_y)

[0 0 1 ... 1 1 0]
Accuracy: 0.4780


# 測定！

In [52]:
pred_y = pd.read_csv('submission.csv', header=None)
final_metrics = calculate_accuracy(pred_y, true_y)

[0 0 1 ... 1 1 0]
Accuracy: 0.9855


# sample submissionのidの違和感からidリークに気がついた方へ

In [63]:
import numpy as np
leak_submission = pd.concat([sample_submission.iloc[:,0], pd.Series(np.where(sample_submission.iloc[:, 0].values < 21417, 0, 1))], axis=1) 

In [65]:
calculate_accuracy(leak_submission, true_y)

[0 0 1 ... 1 1 0]
Accuracy: 0.5251


np.float64(0.5251122907309106)

In [75]:
# リーク、、、、あんまり関係ないっぽい、、、、裏テーマなかった、、、